# 🔄 Silver - tb_custo_chamado (SRC)

**Camada**: Silver  
**Catálogo**: v_credit  
**Tabelas de Saída**:
- `tb_custo_chamado` → Registros válidos
- `tb_custo_chamado_invalidos` → Registros rejeitados (auditoria)

**Origem**: `v_credit.bronze.custos`

**Objetivo**: Limpar e padronizar custos associados aos chamados, convertendo valores monetários e garantindo integridade.

## 📋 Imports e Configuração

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

## 1️⃣ Leitura da Bronze

In [0]:
TABLE_SOURCE = "v_credit.bronze.custos"
TABLE_TARGET_VALIDOS = "v_credit.silver.tb_custo_chamado"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_custo_chamado_invalidos"

df_bronze = spark.table(TABLE_SOURCE)

total_bronze = df_bronze.count()
print(f"\n✅ Total de registros lidos da Bronze: {total_bronze}")

## 2️⃣ Limpeza e Padronização

In [0]:
df_limpo = (
    df_bronze
    .select(
        F.col("id_custo").cast("bigint").alias("cd_custo"),
        F.col("id_chamado").cast("bigint").alias("cd_chamado"),
        F.expr("""
            try_cast(
                replace(
                    regexp_replace(custo, '[^0-9,.-]', ''), 
                    ',', '.'
                ) as decimal(10,8)
            )
        """).alias("vl_custo"),
        # Mantém estritamente o timestamp da origem (Bronze)
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("origem"), F.lit("sistema_custos")).alias("dc_origem")
    )
    .dropDuplicates(["cd_custo"])
    .withColumn("data_processamento", F.current_timestamp())
)

total_limpo = df_limpo.count()
print(f"✅ Dados limpos e padronizados: {total_limpo} registros")

## 3️⃣ Regras de Validação e Qualidade

In [0]:
df_validacao = (
    df_limpo
    # Flag 1: ID do custo obrigatório
    .withColumn("flag_id_valido", F.col("cd_custo").isNotNull())
    # Flag 2: Vínculo com chamado obrigatório
    .withColumn("flag_chamado_valido", F.col("cd_chamado").isNotNull())
    # Flag 3: Valor deve ser numérico válido e maior que zero
    .withColumn(
        "flag_valor_valido", 
        (F.col("vl_custo").isNotNull()) & (F.col("vl_custo") >= 0)
    )
    # Validação Geral
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_chamado_valido") &
            F.col("flag_valor_valido"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

print("✅ Regras de validação aplicadas")

## 4️⃣ Separação de Dados Válidos e Inválidos

In [0]:
# Separação dos DataFrames
df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")

df_invalidos = (
    df_validacao
    .filter(F.col("flag_qualidade") == "ERRO")
    .withColumn(
        "motivo_rejeicao",
        F.concat_ws(
            "; ",
            F.when(~F.col("flag_id_valido"), F.lit("cd_custo NULL")),
            F.when(~F.col("flag_chamado_valido"), F.lit("cd_chamado NULL")),
            F.when(~F.col("flag_valor_valido"), F.lit("vl_custo inválido ou <= 0"))
        )
    )
    .withColumn("dt_validacao", F.lit(processing_timestamp))
    .drop(
        "flag_id_valido",
        "flag_chamado_valido",
        "flag_valor_valido",
        "data_processamento"
    )
)

# Contagens necessárias para o MERGE e logs
total_validos = df_validos.count()
total_invalidos = df_invalidos.count()

print(f"ℹ️ Resumo: {total_validos} válidos | {total_invalidos} inválidos")

## 5️⃣ Gravação: MERGE de Registros Válidos

In [0]:
if total_validos > 0:
    # Criar temp view
    df_validos.createOrReplaceTempView("temp_custo_validos")
    
    print(f"🔄 Executando MERGE de {total_validos} registros válidos em {TABLE_TARGET_VALIDOS}...")
    
    # MERGE SQL - Chave: cd_custo
    spark.sql(f"""
        MERGE INTO {TABLE_TARGET_VALIDOS} AS target
        USING temp_custo_validos AS source
        ON target.cd_custo = source.cd_custo
        
        WHEN MATCHED THEN
            UPDATE SET
                target.cd_chamado = source.cd_chamado,
                target.vl_custo = source.vl_custo,
                target.dt_ingestion = source.dt_ingestion,
                target.dc_origem = source.dc_origem
        
        WHEN NOT MATCHED THEN
            INSERT (
                cd_custo,
                cd_chamado,
                vl_custo,
                dt_ingestion,
                dc_origem
            )
            VALUES (
                source.cd_custo,
                source.cd_chamado,
                source.vl_custo,
                source.dt_ingestion,
                source.dc_origem
            )
    """)
    
    print(f"✅ MERGE concluído com sucesso!")
else:
    print("⚠️  Nenhum registro válido para inserir na Silver")

## 6️⃣ Gravação: APPEND de Registros Inválidos

In [0]:
if total_invalidos > 0:
    print(f"📝 Enviando {total_invalidos} registro(s) inválido(s) para auditoria...")
    
    # APPEND (acumula histórico de rejeições)
    (df_invalidos.write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_TARGET_INVALIDOS))
    
    print(f"✅ Registros inválidos gravados em {TABLE_TARGET_INVALIDOS}")
else:
    print("🎉 Nenhum registro inválido - 100% de qualidade!")